In [14]:
import urllib
from urllib import request
data_jd = request.urlopen("https://cn.bing.com/").read().decode("utf-8", "ignore")
print(len(data_jd))

6530


In [15]:
import re
pat = "<title>(.*?)</title>"  # 表达式
re.compile(pat, re.S).findall(data_jd)

['必应']

In [16]:
# 爬到内存中
data_csdn = request.urlopen("https://www.csdn.net").read().decode("utf-8", "ignore")
re.compile(pat, re.S).findall(data_csdn)

['CSDN - 专业开发者社区']

In [19]:
# 将直接爬取的数据保存到硬盘中
url_jd = "https://baidu.com/"
save_path = "./jd.html"
request.urlretrieve(url_jd, filename=save_path)

('./jd.html', <http.client.HTTPMessage at 0x22e590fe190>)

In [22]:
# 浏览器伪装
opener = request.build_opener()  # 场景opener对象
user_agent_key = "User-Agent"
user_agent_value = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
user_agent_info = (user_agent_key, user_agent_value)
opener.addheaders = [user_agent_info]  # 修改头信息
request.install_opener(opener)  # 将opener对象设置为全局有效
data = request.urlopen(url_jd).read().decode("utf-8", "ignore")
print(len(data))

403904


In [ ]:
# 用户代理池
# 多个浏览器的标识构成用户代理池， 在使用时随机的选择一个去访问， 以防被反爬
import random
user_agent_pools = [
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.5 (KHTML, like Gecko) Chrome/4.0.249.0 Safari/532.5",
    "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.9 (KHTML, like Gecko) Chrome/5.0.310.0 Safari/532.9",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Chrome/7.0.514.0 Safari/534.7",
    "Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.601.0 Safari/534.14",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/10.0.601.0 Safari/534.14",
]
user_agent_key = "User-Agent"
user_agent_value = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
user_agent_info = (user_agent_key, user_agent_value)

url_jd = "https://www.jd.com"


def user_agent_func(pools, user_agent):
    opener = request.build_opener()
    pool_choice = random.choice(pools)
    user_agent = ("User-Agent", pool_choice)
    opener.addheaders = [user_agent]
    request.install_opener(opener)
    print("当前使用的用户代理为： ", pool_choice)

# 每爬两次换一次用户代理
for i in range(0, 6):
    if i % 1 == 0:
        user_agent_func(user_agent_pools, user_agent_info)
    data_jd = urllib.request.urlopen(url_jd).read().decode("utf-8", "ignore")
    print(len(data_jd))

In [26]:
# 分析流程
## 1、取什么数据？
## 内容字段：<div class="main-text">
'''
<div class="content">
<span>
我姐跟我开视频聊天，抱怨她儿子：“你不知道，自从在家上网课，他每天太累了，累的我都心疼！”我大外甥马上过来跟我撒娇：“小姨，我妈都说我累，你是不是得奖励奖励我？”我姐瞬间把手机放下，我盯着天花板听着那边我姐揍娃的动静，一边揍还一边叨叨：“你可不就累坏了么，一整天不是找课本就是找本子，还间歇性找笔找眼镜，还想要奖励
..."
</span>

</div>
'''
## 不同页码的网络地址及其规律
# 2： https://ww.qiushibaike.com/text/page/2
## 构造网址
total_page = 1
for i in range(0, total_page):
    user_agent_func(user_agent_pools, user_agent_info)
    page_url = "https://www.qiushibaike.com/text/page/"+str(i+1)+"/"
    try:
        data = request.urlopen(page_url).read().decode("utf-8", "ignore")
        pat = '<div class="content">.*?<span>(.*?)</span>.*?</div>'
        rst = re.compile(pat, re.S).findall(data)
        for j in range(0, len(rst)):
            print(rst[j])
            print('----')
    except Exception as error:
        print(error)

当前使用的用户代理为：  Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Chrome/7.0.514.0 Safari/534.7
<urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>
